In [7]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
from scipy.integrate import quad
import scipy.integrate as integrate
from numpy import log, exp, sqrt, pi
from scipy.optimize import least_squares

In [16]:
# -------
# DEFINE CUSTOM FUNCTION
# -------
def heston_char_func(phi, T, r, kappa, theta, sigma, rho, v0, S0):
    """
    Heston characteristic function for ln(S_T).
    phi: integration variable (complex)
    T: time to maturity
    r: risk-free rate
    kappa: mean reversion rate of variance
    theta: long-term variance level
    sigma: volatility of volatility
    rho: correlation between Brownian increments
    v0: initial variance
    S0: initial underlying price
    """
    # Helper variables
    i = complex(0.0, 1.0)
    # For P1 and P2 we have slightly different b_j parameters
    # We will handle both cases (j=1,2) by passing appropriate j.
    def characteristic_function(phi, j=1):
        # Parameters differ between j=1 and j=2
        if j == 1:
            u = 0.5
            b = kappa - rho*sigma
        else:  # j = 2
            u = -0.5
            b = kappa
        
        d = np.sqrt((rho*sigma*i*phi - b)**2 - sigma**2*(2*u*i*phi - phi**2))
        g = (b - rho*sigma*i*phi + d)/(b - rho*sigma*i*phi - d)
        
        # C(t,T) and D(t,T) terms
        # Using C and D as standard notation from original Heston derivation
        exp_dT = np.exp(d*T)
        G = (1 - g*exp_dT)/(1 - g)
        C = (r*i*phi*T) + (kappa*theta/sigma**2)*((b - rho*sigma*i*phi + d)*T - 2*np.log(G))
        D = ((b - rho*sigma*i*phi + d)/sigma**2)*((1 - exp_dT)/(1 - g*exp_dT))
        
        return np.exp(C + D*v0 + i*phi*log(S0))
    
    return characteristic_function

def P_j(j, S0, K, T, r, char_func):
    """
    Compute P_j = 1/2 + 1/pi * Integral_0^inf Re[ e^{-i phi ln(K)} f_j(phi)/(i phi) ] dphi
    j: 1 or 2 corresponding to P1 or P2
    """
    i = complex(0.0, 1.0)
    # Integrand for P_j
    def integrand(phi):
        phi_complex = phi + 0.0*i
        f_j = char_func(phi_complex, j=j)
        # Note: f_j is the characteristic function times e^{i phi ln(S0)} already included in char_func definition.
        # The integrand: Re[ e^{-i phi ln(K)} * f_j(phi) / (i phi) ]
        numerator = np.exp(-i*phi_complex*log(K)) * f_j
        return np.real(numerator/(i*phi_complex))
    
    # Integration from 0 to infinity
    # In practice, choose a suitable upper limit. For demonstration, we pick 200.
    # You may need to adjust this limit and possibly use a better integration method.
    phi_max = 200.0
    integral_val, _ = integrate.quad(integrand, 0, phi_max)
    
    return 0.5 + (1.0/pi)*integral_val

def heston_price_call(S0, K, T, r, kappa, theta, sigma, rho, v0):
    """
    Compute the Heston model price for a European call using the Fourier integral.
    """
    char_func = heston_char_func(phi=0, T=T, r=r, kappa=kappa, theta=theta, sigma=sigma, rho=rho, v0=v0, S0=S0)
    # Here char_func returned is actually the function factory. We need to define phi inside P_j calls.
    # Redefine char_func to a lambda that takes phi:
    def cf(phi, j=1):
        return heston_char_func(phi, T, r, kappa, theta, sigma, rho, v0, S0)(phi, j=j)
    
    p1 = P_j(1, S0, K, T, r, cf)
    p2 = P_j(2, S0, K, T, r, cf)
    discount = np.exp(-r*T)
    call_price = S0 * p1 - K * discount * p2
    return call_price

def call_put_parity_put_price(C, S0, K, r, T):
    """
    Given C (call price), S0 (spot price), K (strike), r (risk-free rate), and T (time to maturity),
    this function returns the implied put price:
    P = C - S0 + K * exp(-rT)
    """
    return C - S0 + K * np.exp(-r * T)

def compute_option_price(S0, K, T, r, kappa, theta, sigma, rho, v0, option_type='call'):
    """
    Compute the Heston model price for a European call using the Fourier integral.
    """
    char_func = heston_char_func(phi=0, T=T, r=r, kappa=kappa, theta=theta, sigma=sigma, rho=rho, v0=v0, S0=S0)
    # Here char_func returned is actually the function factory. We need to define phi inside P_j calls.
    # Redefine char_func to a lambda that takes phi:
    def cf(phi, j=1):
        return heston_char_func(phi, T, r, kappa, theta, sigma, rho, v0, S0)(phi, j=j)
    
    p1 = P_j(1, S0, K, T, r, cf)
    p2 = P_j(2, S0, K, T, r, cf)
    discount = np.exp(-r*T)
    call_price = S0 * p1 - K * discount * p2
    if option_type == 'call':
        return call_price
    elif option_type == 'put':
        return call_put_parity_put_price(call_price, S0, K, r, T)

In [19]:
# -------
# EXAMPLE USAGE
# -------
S0 = 100.0      # Current underlying price
K = 100.0       # Strike price
T = 1.0         # 1 year to maturity
r = 0.05        # 5% risk-free rate
kappa = 2.0     # Mean reversion speed
theta = 0.2**2  # Long-run variance
sigma = 0.5     # Vol of vol
rho = -0.5      # Correlation
v0 = 0.15**2    # Initial variance

price = heston_price_call(S0, K, T, r, kappa, theta, sigma, rho, v0)
print("Heston model call price:", price)

Heston model call price: 9.436218927506175


In [21]:
# -------
# GENERATING MARKET DATA WITH KNOWN PARAMETERS
# -------

S0 = 100.0      # Current underlying price
r = 0.05        # 5% risk-free rate
kappa = 2.0     # Mean reversion speed
theta = 0.2**2  # Long-run variance
sigma = 0.5     # Vol of vol
rho = -0.5      # Correlation
v0 = 0.15**2    # Initial variance

market_data = list()
for K in np.linspace(start=90, stop=110, num=5):
    for T in np.linspace(start=1/250, stop=1, num=5):
        market_data.append((K, T, compute_option_price(S0, K, T, r, kappa, theta, sigma, rho, v0, option_type='call')))   

In [23]:
# -------
# DEFINE RESIDUAL FUNCTION
# -------
def calibration_error(params, market_data, S0, r):
    kappa, theta, sigma, rho, v0 = params
    errors = []
    for K, T, market_price in market_data:
        model_price = compute_option_price(S0, K, T, r, kappa, theta, sigma, rho, v0, option_type='call')   
        errors.append(model_price - market_price)
    return np.array(errors)

In [25]:
# -------
# EXPLICIT PARAMETERS
# -------
S0 = 100.0      # Current underlying price
r = 0.05        # 5% risk-free rate

# Initial guess for parameters
initial_guess = [1.5, 0.04, 0.4, -0.7, 0.04]

# Bounds for the parameters (just an example)
bounds = (
    (0.001, 0.0001, 0.0001, -0.9999, 0.0001),
    (10.0, 1.0, 5.0, 0.9999, 1.0)
)

# Use least_squares or minimize
result = least_squares(
    calibration_error,
    initial_guess,
    bounds=bounds,
    args=(market_data, S0, r),
    verbose=2
)

est_kappa, est_theta, est_sigma, est_rho, est_v0 = result.x
print(f'True kappa {kappa}, Estimated kappa {est_kappa}')
print(f'True theta {theta}, Estimated theta {est_theta}')
print(f'True sigma {sigma}, Estimated sigma {est_sigma}')
print(f'True rho {rho},     Estimated rho   {est_rho}')
print(f'True v0 {v0},       Estimated est   {est_v0}')

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         4.9659e+00                                    7.35e+01    
       1              2         3.6913e-01      4.60e+00       2.65e-01       7.58e+00    
       2              3         5.6837e-02      3.12e-01       3.63e-01       1.99e+00    
       3              4         2.8670e-03      5.40e-02       1.20e-01       2.88e-01    
       4              5         1.2351e-05      2.85e-03       2.17e-02       1.05e-01    
       5              6         1.4036e-07      1.22e-05       3.36e-02       8.64e-02    
       6              7         1.8787e-14      1.40e-07       8.79e-05       3.52e-06    
       7              8         7.4959e-23      1.88e-14       2.79e-06       1.96e-09    
`gtol` termination condition is satisfied.
Function evaluations 8, initial cost 4.9659e+00, final cost 7.4959e-23, first-order optimality 1.96e-09.
True kappa 2.0, Estimated kappa 2